In [ ]:
from __future__ import print_function

"""
A script to run cqed-rhf calculation of pyrimidine in the def2-TZVP basis
"""

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [ ]:

# geometry for pyrimidine from Ryan's ORCA calculation
molstr = """

0 1
C     -3.138301    1.766784    0.005631
C     -2.051429    2.626262   -0.001165
C     -0.789779    2.053301   -0.001873
N     -0.631660    0.734101    0.003549
C     -1.729565   -0.011689    0.008655
N     -2.974582    0.448259    0.009994
H     -4.159547    2.136475    0.004056
H     -2.182537    3.700840   -0.006430
H      0.112609    2.657688   -0.007194
H     -1.597993   -1.090027    0.012382
no_reorient
symmetry c1
"""

# options dict
#options_dict = {'basis': 'def2-TZVP',
#               'save_jk': True, 
#               'scf_type': 'pk',
#               'e_convergence' : 1e-8,
#               'd_convergence' : 1e-7}

options_dict = {'basis': 'def2-TZVP',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7}

# to verify this geometry is optimal, you can run the block below
#psi4.set_options(options_dict)
#mol = psi4.geometry(molstr)

#opt_geometry = mol.geometry()
#print(opt_geometry.np*0.529)

Run the calculations below to compute the CQED-RHF energy for various polarizations and
without coupling to a photon.  

Run the cell below to decompose the changes in CQED-RHF energy to the different 1- and 2-electron
components reported in Table I of McTague and Foley.

In [ ]:
# different magnitudes of the lambda vector
l = 0.1


lam_y = np.array([0, l, 0])
lam_xy = np.array([l/np.sqrt(2), l/np.sqrt(2), 0])
    
# results for y-polarized
cqed_dict_y = cqed_rhf(lam_y, molstr, options_dict)
# results for z-polarized

F = cqed_dict_y["PF FOCK MATRIX"]
S = np.array(cqed_dict_y["OVERLAP MATRIX"])

f_file = open('lambda_y_0.1_pyrimidine.out', 'w')
print(F'Fock Matrix for lambda_y = {l}', file=f_file)
for i in range(0,len(F)):
    for j in range(i, len(F)):
        F_val = F[i,j]
        print(F' {i} {j} {F_val}', file=f_file)
        
print(F'Overlap Matrix for lambda_y = {l}', file=f_file)
for i in range(0,len(S)):
    for j in range(i, len(S)):
        F_val = S[i,j]
        print(F' {i} {j} {F_val}', file=f_file)

f_file.close()

In [ ]:
print(cqed_dict_y["CQED-RHF DIPOLE MOMENT"])

In [ ]:
F = cqed_dict_y["PF FOCK MATRIX"]
S = np.array(cqed_dict_y["OVERLAP MATRIX"])

f_file = open('no_lambda_pyrimidine.out', 'w')
print(F'Fock Matrix for lambda_y = {l}', file=f_file)
for i in range(0,len(F)):
    for j in range(i, len(F)):
        F_val = F[i,j]
        print(F' {i} {j} {F_val}', file=f_file)
        
print(F'Overlap Matrix for lambda_y = {l}', file=f_file)
for i in range(0,len(S)):
    for j in range(i, len(S)):
        F_val = S[i,j]
        print(F' {i} {j} {F_val}', file=f_file)

f_file.close()

In [ ]:

#S = np.array(cqed_dict_y["OVERLAP MATRIX"])

In [ ]:
count = 0
for i in range(0,6):
    print(F'{count} \t')
    count += 1
    for j in range(0, 35):
        Sv = S[i, j]
        print(F' {Sv:.6f} ')
    